In [ ]:
#!/usr/bin/env python
# coding: utf-8
# Autor: Ashuin Sharma (A35029)
# Tarea 3.
# Github Repo: https://github.com/hashcr/gis-tarea3/blob/main/Tarea3.ipynb

import requests
from owslib.wfs import WebFeatureService
import pandas as pd
import geopandas as gpd
import folium
import plotly.express as px

# 1. Cargando los datos de Cantones y Red Vial.

In [ ]:
# Cargar cantones y redvial GeoJSON de SNIT en GeoDataFrame
print("Descargando capas de cantones y vias..")

# Carga de registros de Cantones en un dataframe de geopandas
cantones = gpd.read_file("cantones.geojson")

# Carga de registros de RedVial en un dataframe de geopandas
redvial = gpd.read_file("redvial.geojson")
print("Completado.")

# 2. Calculando la intereseccion entre Red Vial y Cantones

In [ ]:
# Filtro solo las columnas que necesito
redvial = redvial[["categoria", "geometry"]]
cantones = cantones[["cod_canton", "canton", "geometry", "area"]]
# Corregimos el valor de area del canton a Km2
cantones["area"] = cantones["geometry"].area / 1000000
# Conseguir la interseccion de Cantones y Red Vial
redvial_cantones = redvial.overlay(cantones, how="intersection")

In [ ]:
# Agregar colummna de longitud de cada intersect
redvial_cantones["longitud_intersect"] = redvial_cantones['geometry'].length/1000

In [ ]:
# Agrupo por canton y categoria sumando las intersecciones.
rv_cant_cat_agrupado = redvial_cantones.groupby(["cod_canton","canton","area","categoria"])["longitud_intersect"].sum()
# Convierto a Data Frame
rv_cant_cat_agrupado = rv_cant_cat_agrupado.reset_index()

# Calculo de la Longitud Total
#Agrupoamos ahora por canton para que sume todas las categorias y quedemos con un DataFrame con 82 cantones
rv_cant_agrupado = rv_cant_cat_agrupado.groupby(["cod_canton"])["longitud_intersect"].sum()
# Convertir a DataFrame
rv_cant_agrupado = rv_cant_agrupado.reset_index()

# Ahora unimos a la tabla de cantones con un Join para agregarle la columna de Longitud total
tabla_cant = cantones.join(rv_cant_agrupado.set_index('cod_canton'), on='cod_canton', rsuffix='_b')
tabla_cant.rename(columns = {'longitud_intersect': 'Longitud Total', 'area': 'Area', 'canton': 'Canton'}, inplace = True)

# Calculo de la Densidad Total
tabla_cant["Densidad Total"] = tabla_cant["Longitud Total"] / tabla_cant["Area"] ;

categorias = ["Autopista", "Carretera Pavimento Dos Vias o Mas", "Carretera Pavimento Una Via", "Carretera Sin Pavimento Dos Vias", "Camino de Tierra"]

# Calculamos auto
for cat in categorias:
    temp_join = rv_cant_cat_agrupado.loc[rv_cant_cat_agrupado["categoria"] == cat.upper()][["cod_canton", "longitud_intersect"]]
    tabla_cant = tabla_cant.join(temp_join.set_index('cod_canton'), on='cod_canton', rsuffix='_b')
    tabla_cant.rename(columns = {'longitud_intersect': cat}, inplace = True)
    tabla_cant[cat] = tabla_cant[cat].fillna(0)

tabla_cant = tabla_cant.sort_values("cod_canton", ascending=[True])

# 1. Tabla de cantones

In [ ]:
# Tarea 3. Item 1. 
# Tabla de Cantones

tabla_cant[["cod_canton", "Canton", "Autopista", "Carretera Pavimento Dos Vias o Mas" , "Carretera Pavimento Una Via",
            "Carretera Sin Pavimento Dos Vias", "Camino de Tierra", "Longitud Total", "Densidad Total"]]

# 2. Top 15 Cantones

In [ ]:
# Instanciamos el gráfico de plotly y definios X como Cantón , Y como Longitud.
# Y los colores los definirán las 5 categorias de vía.

# Dataframe filtrado con los top 15 cantones como mayor red vial, para usar en graficación
tabla_cant_grafico = tabla_cant.sort_values("Longitud Total", ascending=[False]).head(15)

fig = px.bar(tabla_cant_grafico, 
             x='Canton', 
             y=["Autopista", "Carretera Pavimento Dos Vias o Mas" , "Carretera Pavimento Una Via",
                "Carretera Sin Pavimento Dos Vias", "Camino de Tierra"], 
             title="Top 15 cantones con mayor longitud total de red vial (tipo de vía).",
             labels={
                "value": "Longitud vial (Km)", "variable": "Tipos de vía", "Canton" : "Cantón"
            })

fig.show()

# 3. Gráfico de Pastel. Distribución Total de Red Vial por Cantones.

In [ ]:
# Dataframe filtrado con los top 15 cantones como mayor red vial, para usar en graficación
tabla_cant_filtro = tabla_cant.sort_values("Longitud Total", ascending=[False]).head(15)

# Obtengo el valor del ultimo canton
nth_row = 14
tabla_cant_filtro = tabla_cant_filtro.iloc[nth_row]
ultimo = tabla_cant_filtro["Longitud Total"]

# Ahora traigo todos pero ordenados y modifico la variable canton de los menores al 15vo.
tabla_cant_pie = tabla_cant.sort_values("Longitud Total", ascending=[False])
tabla_cant_pie.loc[tabla_cant_pie['Longitud Total'] < ultimo, 'Canton'] = 'Otros'

# Creacion del Pie Chart
fig = px.pie(tabla_cant_pie, values='Longitud Total', names='Canton', title='Gráfico de Pastel. Distribución Total de Red Vial por Cantones.')
fig.show()

## 4. Mapa de coropletas Densidad Vial de Costa Rica.

In [ ]:


# Creación del mapa base
m = folium.Map(location=[9.8, -84], 
               tiles='CartoDB positron',
               control_scale=True,
               zoom_start=8)

folium.Choropleth(
    name="Densidad Vial",
    geo_data=cantones,
    data=tabla_cant,
    columns=['cod_canton', 'Densidad Total'],
    bins=7,
    key_on='feature.properties.cod_canton',
    fill_color='Reds', 
    fill_opacity=0.8, 
    line_opacity=1,
    legend_name='Densidad vial por cantón',
    width=800, height=700,
    smooth_factor=0).add_to(m)

# Añadir capa de Red Vial
folium.GeoJson(data=redvial, name='Red vial').add_to(m)

# Control de capas
folium.LayerControl().add_to(m)

# Despliegue del mapa
m